# Simple Procedural Content Generation for Space Engineers

Create functional spaceships in Space Engineers by PCG using the [SE API](https://github.com/iv4xr-project/iv4xr-se-plugin).

Additional information for SE can be found [here](https://github.com/iv4xr-project/iv4xr-se-plugin/tree/main/JvmClient#basic-space-engineers-engine-info).

SE API calls are listed [here](https://github.com/iv4xr-project/iv4xr-se-plugin/blob/main/Source/Ivxr.SePlugin/Communication/AustinJsonRpcSpaceEngineers.cs).

**NOTE**: Have the game *running*; leaving the game simply paused breaks the API (no calls are accepted). *Suggestion*: Use windowed mode, press `Start` after the scenario is loaded and return to this notebook.

Import required packages

In [ ]:
import ast
import json
import random
import os
import socket
import time

from enum import Enum
from typing import List, Optional, Dict, Any, Tuple

## API Interfacing

Variables for the API:

In [ ]:
host = 'localhost'
port = 3333

API common structs/classes

In [ ]:
def vec2(x: int,
         y: int) -> Dict[str, int]:
    return {
        "X": x,
        "Y": y
    }

def vec3(x: int,
         y: int,
         z: int) -> Dict[str, int]:
    return {
        "X": x,
        "Y": y,
        "Z": z
    }

def vec2f(x: float,
         y: float) -> Dict[str, float]:
    return {
        "X": x,
        "Y": y
    }

def vec3f(x: float,
         y: float,
         z: float) -> Dict[str, float]:
    return {
        "X": x,
        "Y": y,
        "Z": z
    }

def sum_vecs(a: Dict[str, Any],
             b: Dict[str, Any]) -> Dict[str, Any]:
    return {
        "X": a["X"] + b["X"],
        "Y": a["Y"] + b["Y"],
        "Z": a["Z"] + b["Z"],
    }

class Orientation(Enum):
    UP = vec3f(0., 1., 0.)
    DOWN = vec3f(0., -1., 0.)
    RIGHT = vec3f(1., 0., 0.)
    LEFT = vec3f(-1., 0., 0.)
    FORWARD = vec3f(0., 0., -1.)
    BACKWARD = vec3f(0., 0., 1.)

Character position to camera position offset

In [ ]:
character_camera_dist = vec3f(0., 1.6369286, 0.)

Block class.

*Possible Todo*: blocks have different side sizes (small: 0.5, large: 2.5 etc... see [here](https://github.com/iv4xr-project/iv4xr-se-plugin/tree/main/JvmClient#small-vs-large-cube-blocks)) so it may be useful to have a way to allign them automatically regardless of size.

In [ ]:
class Block:
    def __init__(self,
                 block_type: str,
                 position: Dict[str, Any],
                 orientation_forward: Orientation = Orientation.FORWARD,
                 orientation_up: Orientation = Orientation.UP,
                 relative_position: bool = True):
        self.block_type = block_type
        self.relative_position = relative_position
        self.position = position
        self.orientation_forward = orientation_forward
        self.orientation_up = orientation_up

API interfacing methods

In [ ]:
def generate_json(method: str,
                  params: Optional[List[Any]] = None) -> Dict[str, Any]:
    return {
        "jsonrpc": "2.0",
        "method": method,
        "params": params if params else [],
        "id": random.getrandbits(16)
    }

def compactify_jsons(jsons: List[Dict[str, Any]]) -> List[str]:
    """ JSONs should be single line & compact formatted before calling the API. (@Karel Hovorka)"""
    compacted_jsons = ''
    for j in jsons:
        compacted_jsons += json.dumps(obj=j,
                               separators=(',', ':'),
                               indent=None)
        compacted_jsons += '\r\n'

    return compacted_jsons

# Modified from https://www.binarytides.com/receive-full-data-with-the-recv-socket-function-in-python/
def recv_with_timeout(s: socket.socket,
                      timeout: int = 2):
    #make socket non blocking
    s.setblocking(0)
    #total data partwise in an array
    total_data = [];
    #beginning time
    begin = time.time()
    while True:
        # timeout termination
        if total_data and time.time() - begin > timeout:
            break
        # wait to get data
        elif time.time() - begin > timeout * 2:
            break
        # recv
        try:
            data = s.recv(8192).decode("utf-8")
            if data:
                # early break if socket returns the same data
                if len(total_data) > 1 and data == total_data[-1]:
                    break
                # append new data
                total_data.append(data)
                # change the beginning time for measurement
                begin = time.time()
            else:
                #sleep to indicate a gap
                time.sleep(0.1)
        except:
            # note: Exceptions are thrown due to socket not reading anything before and after data is passed
            pass
    # join all parts to make final string
    return ''.join(total_data)

def call_api(jsons: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    # open socket for communication
    s = socket.socket(family=socket.AF_INET,
                      type=socket.SOCK_STREAM)
    s.connect((host, port))    
    # send methods as compacted JSON bytearray
    s.sendall(compactify_jsons(jsons=jsons).encode('utf-8'))
    # get response
    res = recv_with_timeout(s)
    # close socket
    s.close()
    # due to TCP streming packets, it's possible some JSON-RPC responses are the same;
    # workaround: identify unique JSON-RPC responses by unique id
    return [json.loads(x) for x in list(set(res.strip().split('\r\n')))]

Simple test:

In [ ]:
jsons = [
    generate_json(method="Character.TurnOnJetpack"),
    generate_json(method="Character.MoveAndRotate",
                  params={
                    "movement": vec3(10, 0, -1),
                    "rotation3": vec2f(0., 20.),
                    "roll": 0.}
                 )
]

# print(call_api(jsons=jsons))

## Game speed

We can toggle the game speed by togglin the game's frame limit.

**NOTE**: Switch between `GameMode.PLACING` (fast) and `GameMode.EVALUATING` (slow).

In [ ]:
class GameMode(Enum):
    PLACING = False
    EVALUATING = True

def toggle_gamemode(mode: GameMode) -> None:
    # "we plan to change the API and rename this function in near future" (@Karel Hovorka)
    call_api(jsons=[generate_json(method="Admin.SetFrameLimitEnabled",
                                  params=[mode.value])])

In [ ]:
toggle_gamemode(mode=GameMode.EVALUATING)

## Block placement

[Blocks DefintionIDs](https://github.com/iv4xr-project/iv4xr-se-plugin/tree/main/JvmClient#definitionid-id) are used to specify which block to place. Here we either ask the API directly to provide us the list of available DefintionIDs or we can load the .JSON which contains them (faster in future runs).

**NOTE**: This cell produces the global variable `block_definitions`, a map between the block type and the API-supported ID to use in the request `params` field.

In [ ]:
if not os.path.exists('./block_definitions.json'):
    # poll API for block definition ids
    jsons = [ 
        generate_json(method="Definitions.BlockDefinitions")
        ]
    res = json.loads(call_api(jsons=jsons))
    # transform to map of type:id
    block_definitions = {v['Type']:v for v in [entry['DefinitionId'] for entry in res['result']]}
    with open('./block_definitions.json', 'w') as f:
        json.dump(block_definitions, f)
else:
    with open('./block_definitions.json', 'r') as f:
        block_definitions = json.load(f)

Place structure methods.

`place_structure` does batch placement, `place_structure_seq` places each block of the structure sequentially with a time interval between each placement.

In [ ]:
def place_structure(structure):
    # get base position and orientations (player-relative)
    obs = call_api(jsons=[generate_json(method="Observer.Observe")])[0]
    base_position = obs['result']['Position']
    # prepare jsons
    jsons = [generate_json(method="Admin.Blocks.PlaceAt",
                           params={
                               "blockDefinitionId": block_definitions[block.block_type],
                               "position": sum_vecs(base_position, block.position) if block.relative_position else block.position,
                               "orientationForward": block.orientation_forward,
                               "orientationUp": block.orientation_up
                           }) for block in structure]
    # produce structure
    call_api(jsons=jsons)

def place_structure_seq(structure,
                        t: float = 0.1):
    # get base position and orientations (player-relative)
    obs = call_api(jsons=[generate_json(method="Observer.Observe")])[0]
    base_position = obs['result']['Position']
    # make sequential API calls
    for block in structure:
        jsons = [generate_json(method="Admin.Blocks.PlaceAt",
                               params={
                                   "blockDefinitionId": block_definitions[block.block_type],
                                   "position": sum_vecs(base_position, block.position) if block.relative_position else block.position,
                                   "orientationForward": block.orientation_forward,
                                   "orientationUp": block.orientation_up
                                   })]
        # place block
        call_api(jsons=jsons)
        time.sleep(t)

Hardcoded structure (relative positions for each block)

In [ ]:
structure = [
    Block('SmallBlockArmorBlock', vec3(0,0,0), Orientation.FORWARD, Orientation.UP, relative_position=True),
    Block('SmallBlockArmorBlock', vec3(0,0,1), Orientation.FORWARD, Orientation.UP, relative_position=True),
    Block('SmallBlockArmorBlock', vec3(0,0,2), Orientation.FORWARD, Orientation.UP, relative_position=True),
    Block('SmallBlockArmorBlock', vec3(0,1,2), Orientation.FORWARD, Orientation.UP, relative_position=True),
    Block('SmallBlockArmorBlock', vec3(0,2,2), Orientation.FORWARD, Orientation.UP, relative_position=True),
    Block('SmallBlockArmorBlock', vec3(0,2,1), Orientation.FORWARD, Orientation.UP, relative_position=True),
    Block('SmallBlockArmorBlock', vec3(0,2,0), Orientation.FORWARD, Orientation.UP, relative_position=True),
    Block('SmallBlockArmorBlock', vec3(0,1,0), Orientation.FORWARD, Orientation.UP, relative_position=True),
]

Resulting structure should look like this:

![block_placement_example](https://raw.githubusercontent.com/arayabrain/space-engineers-research/main/block-placement/block_placement_example.png?token=AH22MZEXJQQTS4V6RKO633LBRU44M)

In [ ]:
# place_structure(structure)

In [ ]:
structure = [
    Block('LargeBlockSmallGenerator', vec3(0,-2.5,0), Orientation.LEFT, Orientation.DOWN, relative_position=True),
    Block('LargeBlockCockpitSeat', vec3(0,0,0), Orientation.UP, Orientation.RIGHT, relative_position=True),
]

At the moment, the structure spawns but is not functional as the blocks are not part of the same grid, either with `place_structure` or `place_structure_seq`. Waiting to hear back from GoodAI.

In [ ]:
place_structure_seq(structure)